In [ ]:
# setup

scratch = '/pscratch/sd/j/jrobnik/quasars/'
ids = jnp.load(scratch + 'ids.npy') # a random id


def inject_recovery(key, id, hyp):
    
    df = pd.read_csv(scratch + str(id) + '.csv')
    time = jnp.array(df['time'])
    mag = jnp.array(df['mag'])
    mag_err = jnp.array(df['mag_err'])
    cov = psd.covariance(time, psd.drw_kernel(*hyp), mag_err)

    data = irregular_spaced(key, cov)
    null, alternative = psd.nlog_density(time, data, mag_err, freq[0], freq[-1])
    opt = minimize(null, x0 = jnp.log(hyp), method = 'BFGS')
    return jnp.exp(opt.x)


hyp = jnp.array([0.1, 10.])
num_sim = 2
keys = jax.random.split(key, num_sim)
recovered = jnp.array([inject_recovery(keys[i], ids[i], hyp) for i in range(num_sim)])

plt.plot(recovered[:, 0], recovered[:, 1], '.', color= 'teal', label= 'recovered')
plt.plot([hyp[0],], [hyp[1],], '*', markersize = 10, color = 'gold', label = 'injected')
plt.xlabel(r'$\sigma$')
plt.ylabel(r'$\tau$')
plt.yscale('log')
plt.xscale('log')
plt.show()